In [12]:
import json, sys, settings, random
import lasair
from scipy.stats import ttest_ind

## The `meanChange` features 
This simple feature is to filter out lightcurves with a significant change in brightness. It lumps together all 6 wavebands into new and old, then uses
Student's t-test to decide if the mean flux has changed. The meaning of new and old is set here to 10 days.

In [13]:
# go back 10 days and 10 days before that for 2 populations
timeInterval = 10       # days

# go back from this MJD
MJD          = 60655.5  # 2024-12-11 13:00

In [14]:
# the lasair client will be used for pulling all the info about the object
# and for annotating it
L = lasair.lasair_client(settings.API_TOKEN)
selected = 'objects.objectId, sherlock_classifications.classification as class'
tables = 'objects,sherlock_classifications'
conditions = 'ncandgp>20 AND objects.jdmax > %.1f' % (MJD - 2*timeInterval)
max_alert = 20
results = L.query(selected, tables, conditions, limit=max_alert)

### This function uses Student t-test for different in means
The return is a p-value, which is the probability the means are the same old and new.
Thus the smaller the p-value, the more we believe the means are different

In [15]:
def change_of_mean(alert, time_interval):
    # for ZTF, the candidates and noncandiates are mixed in, and the noncandidates have no candid
    sources  = [c for c in alert['candidates'] if 'candid' in c]
    sources.sort(key = lambda c: c['mjd'])

    # time of the most recent candidate
    tlast = sources[-1]['mjd']
    magsnew = []  # between t-time_interval and t
    magsold = []  # between t-2*time_interval and t-time_interval
    for c in sources:
        if c['mjd'] > tlast - 2*time_interval:
#            print(c['mjd'], c['magpsf'])
            if c['mjd'] > tlast - time_interval:
                magsnew.append(c['magpsf'])
            else:
                magsold.append(c['magpsf'])
#    print(objectId, len(sources), len(magsold), len(magsnew))
    if len(magsnew) < 3 or len(magsold) < 3:
        return None
    magsoldmean = sum(magsold)/len(magsold)
    magsnewmean = sum(magsnew)/len(magsnew)

    s = ttest_ind(magsold, magsnew)
    return (s.pvalue, len(magsold), len(magsnew), magsnewmean-magsoldmean)

### Fetch the lightcurves of the found objects

In [16]:
for row in results:
    objectId = row['objectId']
    alert = L.objects([objectId])[0]
    ret = change_of_mean(alert, timeInterval)
    if ret: 
        (pvalue, n_old, n_new, new_minus_old) = ret
        if pvalue < 0.05:
            print('https://lasair-ztf.lsst.ac.uk/objects/%s %3s %8.3f (%d %d %.1f)' % \
                  (objectId, row['class'], pvalue, n_old, n_new, new_minus_old))

https://lasair-ztf.lsst.ac.uk/objects/ZTF17aaaikoz  CV    0.027 (11 15 0.5)
